In [1]:
# Install Additional Python Libraries
!pip install -r requirements.txt

In [2]:
# Need postgres
# https://mvnrepository.com/artifact/org.postgresql/postgresql
from spark_libs import spark_submit
packages = ["com.databricks:spark-csv_2.11:1.5.0", 
            "org.postgresql:postgresql:42.2.9"]
spark_submit(packages=packages)

Adding environment variable `PYSPARK_SUBMIT_ARGS`
--packages com.databricks:spark-csv_2.11:1.5.0,org.postgresql:postgresql:42.2.9 pyspark-shell


In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [4]:
# get or create Spark session

app_name = "spark-postgres"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [5]:
user_payment_file ="../Resources/user_payment.csv"
user_data_file ="../Resources/user_data.csv"

spark.sparkContext.addFile(user_payment_file)
spark.sparkContext.addFile(user_data_file)

df_payment = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(SparkFiles.get("user_payment.csv"))
df_payment.printSchema()


df_data = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(SparkFiles.get("user_data.csv"))
df_data.printSchema()

root
 |-- billing_id: integer (nullable = true)
 |-- username: string (nullable = true)
 |-- cc_encrypted: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- active_user: boolean (nullable = true)
 |-- street_address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- username: string (nullable = true)



In [12]:
# Postgres credentials
jdbcHostname = "host.docker.internal"
jdbcPort = "5433" # should be 5432 for you
jdbcDatabase = "users"
dialect = "postgresql"
jdbcUsername = "postgres"
jdbcPassword = "changeme"

jdbcUrl = "jdbc:{3}://{0}:{1}/{2}".format(jdbcHostname, jdbcPort, jdbcDatabase, dialect)
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}
# for mysql driver = com.mysql.jdbc.Driver

In [13]:
jdbcUrl

'jdbc:postgresql://host.docker.internal:5433/users'

In [14]:
connectionProperties

{'user': 'postgres', 'password': 'changeme', 'driver': 'org.postgresql.Driver'}

In [15]:
table = "user_data"
mode = "overwrite" # options are: error, append, overwrite

df_data.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [16]:
table = "user_payment"
mode = "overwrite" # options are: error, append, overwrite

df_payment.write.jdbc(jdbcUrl, table, mode, connectionProperties)